In [40]:
import pandas as pd
import networkx as nx
import math

In [4]:
data = pd.read_csv('../data_omim/full_omim_table.txt', delimiter = "\t")
data

,genes,hgnc_synonyms,hgnc_genes,phenotype,phenotypeInheritance,geneMimNumber,phenotypeMimNumber,chromosome,comments
0,AD7CNTP,NaN,NaN,NaN,NaN,607413,NaN,1,NaN
1,ALPQTL2,NaN,NaN,"Alkaline phosphatase, plasma level of, QTL 2",None,612367,612367.0,1,linkage with rs1780324
2,ANON1,NaN,NaN,"Anorexia nervosa, susceptibility to, 1",None,606788,606788.0,1,NaN
3,BCC1,NaN,NaN,"Basal cell carcinoma, susceptibility to, 1",None,605462,605462.0,1,associated with rs7538876
4,BMND3,NaN,NaN,Bone mineral density QTL 3,None,606928,606928.0,1,?another locus at 3p21
...,...,...,...,...,...,...,...,...,...
18266,CDY1|CDY,CDY1|CDY2B,"CDY2B,CDY1",NaN,NaN,400016,NaN,Y,NaN
18267,TTTY3,TTTY3,TTTY3,NaN,NaN,400036,NaN,Y,NaN
18268,GCY|TSY|STA,"GCY|GCY|GCY,EMD","GCY,EMD",NaN,NaN,475000,NaN,Y,NaN
18269,DFNY1,DFNY1,DFNY1,"Deafness, Y-linked 1",None,400043,NaN,Y,NaN


In [13]:
#matrix = loadmat('../data_omim/MimMiner_Exp_AC_T_TXCS_basedonACMESH_filt_RW.mat')
matrix = pd.read_csv('../data_omim/MimMiner_Exp_AC_T_TXCS_basedonACMESH_filt_RW.txt', delimiter = "\t")
matrix

,100050,1,0.0890344,0.361651,0.266263,0.368639,0.197142,0.0678806,0.203997,0.238039,...,0.150,0.0567316,0.0920642,0.0492268,0.178066,0.132249,0.118521,0.157154,0.163718,0.195514
0,100070,0.089034,1.000000,0.087366,0.026785,0.148069,0.233403,0.125448,0.072799,0.089556,...,0.099061,0.000000,0.113751,0.000000,0.122792,0.130740,0.161343,0.031409,0.137056,0.041865
1,100100,0.361651,0.087366,1.000000,0.042354,0.269991,0.112267,0.073997,0.293200,0.170992,...,0.078151,0.000000,0.164024,0.028643,0.173101,0.115605,0.096322,0.000000,0.000000,0.132642
2,100200,0.266263,0.026785,0.042354,1.000000,0.215814,0.189442,0.000000,0.108829,0.093592,...,0.000000,0.125662,0.067406,0.109038,0.113316,0.042975,0.110791,0.351535,0.365798,0.154049
3,100300,0.368639,0.148069,0.269991,0.215814,1.000000,0.232661,0.074826,0.243982,0.297043,...,0.056422,0.070380,0.116400,0.040664,0.203403,0.139057,0.171796,0.128069,0.133476,0.216301
4,100600,0.197142,0.233403,0.112267,0.189442,0.232661,1.000000,0.077624,0.145725,0.214755,...,0.085842,0.107727,0.155793,0.062254,0.171311,0.196672,0.210664,0.195577,0.293568,0.164787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5074,609015,0.132249,0.130740,0.115605,0.042975,0.139057,0.196672,0.159548,0.057489,0.203543,...,0.079654,0.066980,0.054236,0.058119,0.273956,1.000000,0.599939,0.050258,0.053145,0.048767
5075,609016,0.118521,0.161343,0.096322,0.110791,0.171796,0.210664,0.243449,0.063666,0.238993,...,0.066117,0.055783,0.110602,0.048403,0.186326,0.599939,1.000000,0.132196,0.113348,0.068000
5076,609021,0.157154,0.031409,0.000000,0.351535,0.128069,0.195577,0.073371,0.043691,0.094153,...,0.000000,0.146956,0.039533,0.192027,0.079588,0.050258,0.132196,1.000000,0.552150,0.144883
5077,609026,0.163718,0.137056,0.000000,0.365798,0.133476,0.293568,0.000000,0.046156,0.099291,...,0.000000,0.155398,0.041861,0.202934,0.115536,0.053145,0.113348,0.552150,1.000000,0.152830


In [71]:
my_ids_dictionary = dict()
my_rows_dictionary = dict()

True

In [72]:
def get_id_of_node(node):
    if node not in my_ids_dictionary:
        for row in matrix.loc[matrix['100050'] == node].iterrows():
            my_ids_dictionary[node] = row[0]
            return row[0]
    else:
        return my_ids_dictionary[node]

In [73]:
def get_similarity_score(node1, node2):
    node2_id = get_id_of_node(node2)
    
    if (node2_id != None):
        if node1 not in my_rows_dictionary:
            for row in matrix.loc[matrix['100050'] == node1].iterrows():
                my_rows_dictionary[node1] = row[1]
                return row[1][node2_id + 2]
        else:
            return my_rows_dictionary[node1][node2_id + 2]
    return None

In [74]:
print(get_similarity_score(21412, 612124124367))

None


In [76]:
len(data)
G_diseases = nx.Graph()

In [77]:
for i, row in enumerate(data.iterrows()):
    for j, row2 in enumerate(data.iterrows()):
        if i > j and not math.isnan(row[1]['phenotypeMimNumber']) and not math.isnan(row2[1]['phenotypeMimNumber']):
            phenotypeMimNumber1 = int(row[1]['phenotypeMimNumber'])
            phenotypeMimNumber2 = int(row2[1]['phenotypeMimNumber'])
            
            similarity_score = get_similarity_score(phenotypeMimNumber1, phenotypeMimNumber2)
            
            if similarity_score:
                G_diseases.add_node(phenotypeMimNumber1)
                G_diseases.add_node(phenotypeMimNumber2)
                G_diseases.add_edge(phenotypeMimNumber1, phenotypeMimNumber2, weight=similarity_score)
                #print(similarity_score)
    if i % 10 == 0:
        print(i / len(data))

0.0
0.0005473154178753215
0.001094630835750643
0.0016419462536259647
0.002189261671501286
0.0027365770893766077
0.0032838925072519293
0.003831207925127251
0.004378523343002572
0.004925838760877894
0.005473154178753215
0.006020469596628537
0.006567785014503859
0.00711510043237918
0.007662415850254502
0.008209731268129824
0.008757046686005144
0.009304362103880467
0.009851677521755788
0.01039899293963111
0.01094630835750643
0.011493623775381753
0.012040939193257074
0.012588254611132395
0.013135570029007717
0.013682885446883038
0.01423020086475836
0.014777516282633681
0.015324831700509004
0.015872147118384326
0.016419462536259647
0.016966777954134968
0.01751409337201029
0.018061408789885613
0.018608724207760934
0.019156039625636254
0.019703355043511575
0.020250670461386896
0.02079798587926222
0.02134530129713754
0.02189261671501286
0.022439932132888182
0.022987247550763507
0.023534562968638827
0.024081878386514148
0.02462919380438947
0.02517650922226479
0.025723824640140114
0.0262711400580

In [78]:
nx.write_pajek(G_diseases, "../networks/disease_disease_network.net")

In [81]:
#G_diseases.nodes(0)